In [1]:
# import os
# os.chdir("../")

In [2]:
import pandas as pd
from pandas_dedupe import dedupe_dataframe

In [3]:
def filter_out_non_duplicates(df: pd.DataFrame):
    df = df.loc[df.tracking_id.notna(), ['uid', 'tracking_id']].groupby("tracking_id").filter(lambda x: len(x) >= 2)
    df["uid_counts"] = df.loc[df.tracking_id.notna(), ['uid', 'tracking_id']].groupby("tracking_id")["uid"].transform("nunique")
    df = df[~((df.uid_counts.astype(str) == "1"))]
    return df.drop(columns=["uid_counts"])


def groupby_tracking_number_and_uid(df):
    return pd.DataFrame(
        ((tn, id.str.cat(sep=", ")) for tn, id in df.groupby(['tracking_id'])['uid']),
        columns=["tracking_id", "uids"])


def create_clusters(df):
    df = dedupe_dataframe(df, ["uids"])
    return df

"""
3     243e11a7128df89c570314aed55a9dc8  2020-0565-r  2ca005708d4aa4ec982294f764cb71d8
4     14b8b33c301f5580041a965e27d1c2e6  2020-0565-r  4713804b02e3e76dfbca8da1c2dfe710
5     14b8b33c301f5580041a965e27d1c2e6  2020-0565-r  3557a6f8b03ebb802e4ea1d5463b1504
"""


def filter_clusters_for_exact_coaccusals(df):
    df = df[df.duplicated(subset=['cluster_id'], keep=False)]
    return df


def split_rows_with_multiple_uids(df):
    df = (
        df.drop("uids", axis=1)
        .join(
            df["uids"]
            .str.split(", ", expand=True)
            .stack()
            .reset_index(level=1, drop=True)
            .rename("uids"),
            how="outer",
        )
        .reset_index(drop=True)
    )
    return df.rename(columns={"uids": "uid"})

In [6]:
def cluster():
    dfa = pd.read_csv("../data/fuse/allegation.csv")
    dfb = (dfa
        .pipe(filter_out_non_duplicates)
        .pipe(groupby_tracking_number_and_uid)
        .pipe(create_clusters)
        .rename(columns={"cluster id": "cluster_id"})
        .pipe(filter_clusters_for_exact_coaccusals)
        .pipe(split_rows_with_multiple_uids)
    )
    dfa = dfa[["allegation_uid", "tracking_id", "uid"]]
    df = pd.merge(dfa, dfb, on=["uid", "tracking_id"])
    df = df.drop_duplicates(subset=["allegation_uid"])   
    return df.sort_values("cluster_id").drop(columns=["confidence"])

In [7]:
df = cluster()
df

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (3,6,8,12,15,16,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


Importing data ...


c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas_dedupe\utility_functions.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df[i] = df[i].str.replace('[^\w\s\.\-\(\)\,\:\/\\\\]','')
uids : 65ca05f3b63f7d7383662d1275b0804f, 2d430d5918f53d6216f87614beff9ec9, 6c0f5764509f4a2231be815554b0fe27, 2d430d5918f53d6216f87614beff9ec9

uids : 65ca05f3b63f7d7383662d1275b0804f, 2d430d5918f53d6216f87614beff9ec9, 6c0f5764509f4a2231be815554b0fe27

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


Starting active labeling...
